# Example Evaluation Code

This notebook will be very __similar__ to the code I use to evaluate your results - it is provided for __your convenience__ so that you can use it to evaluate your preprocessing results at any time before your __final submission__.

Please note that the results here will __NOT__ be the same as my evaluation results.

Let's start with loading the required packages.

In [1]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

Next you should load __your__ data. In this case, I am using a sample dataset (`GroupX.csv`) which contains 6 predictors (`X1 - X6`) and two target variables (`Y1, Y2`).

Please make sure you change the data to your __OWN__ dataset when using this code.

__NOTE__:
1. Your dataset maybe very different from the sample dataset.
2. Please follow this structure when submitting your dataset.

In [2]:
# what to include (continous)
#data = data[['C1', 'C2', 'C4', 'C5', 'C6', 'T1', 'T2', 'T3', 'T4', 'T5', 'C3_impute', 'C7_impute', 'C3`',
       #'C5`', 'C6`', 'T1prime', 'T3prime', 'T4prime', 'S1prime',
       #'S2prime', 'S3prime','Y1', 'Y2',]]

In [3]:
data = pd.read_csv('all_data_y1y2_seperated.csv', header=0,index_col=0)
#y2 = pd.read_csv('y2_features.csv', header=0, index_col=0)

#data = data[['C1', 'C4', 'C5', 'C6', 'C3_impute', 'C7_impute', 'C5`',
       #'C6`', 'T1prime', 'T2prime', 'T4prime', 'T5prime', 'S1prime', 'S2prime',
       #'S3prime', 'Y1', 'Y2']]
#data = data[['C4', 'T1prime', 'S1prime_log', 'S2prime_log',
#       'S3prime_log', 'C1_norm', 'C5_norm', 'C6_norm', 'C3_norm', 'C7_norm',
#       'T3prime_norm', 'T4prime_norm', 'T2prime_norm', 'T5prime_norm', 'Y1',
#       'Y2']]
data.head()

,C3`,C1,C5,T1prime,S2prime,Y1,Y2
0,1,-0.192492,-0.087268,0.171085,0.058024,-1.0,1.268344
1,0,1.028858,-0.307680,0.233498,0.716621,1.0,-0.788430
2,0,-0.477770,-0.526829,0.119030,-0.953443,1.0,-0.788430
3,0,0.583110,-0.563229,-1.282432,-0.652039,1.0,-0.788430
4,1,-0.566920,-0.273551,-0.593417,0.351143,-1.0,1.268344


In [4]:
# just a test 
cols = list(data.columns)
pred_y1 = data[cols[:-7]]
pred_y1

""
0
1
2
3
4
...
677
678
679
680


In [5]:
# just a test 
cols1 = list(data.columns)
pred_y2 = data[cols[6:]]
pred_y2

,Y2
0,1.268344
1,-0.788430
2,-0.788430
3,-0.788430
4,1.268344
...,...
677,1.268344
678,-0.788430
679,-0.788430
680,1.268344


In [6]:
y1 = data.Y1
y2 = data.Y2

In [7]:
y2.head()

0    1.268344
1   -0.788430
2   -0.788430
3   -0.788430
4    1.268344
Name: Y2, dtype: float64

In [8]:
data.columns

Index(['C3`', 'C1', 'C5', 'T1prime', 'S2prime', 'Y1', 'Y2'], dtype='object')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 0 to 681
Data columns (total 7 columns):
C3`        660 non-null int64
C1         660 non-null float64
C5         660 non-null float64
T1prime    660 non-null float64
S2prime    660 non-null float64
Y1         660 non-null float64
Y2         660 non-null float64
dtypes: float64(6), int64(1)
memory usage: 41.2 KB


In [10]:
data.isna().sum()

C3`        0
C1         0
C5         0
T1prime    0
S2prime    0
Y1         0
Y2         0
dtype: int64

Checking your data types and make sure it follows the data dictionary would be an important step, you can do that using the `.dtypes` attribute.

__NOTE__: all __continuous__ faetures will be in `float64` data type, and all __categorical__ features will be in `int64` data type (given you already coded (per __suggest task \#6__ in the competition document) them).

In [11]:
data.dtypes

C3`          int64
C1         float64
C5         float64
T1prime    float64
S2prime    float64
Y1         float64
Y2         float64
dtype: object

Now you need to specify your targets and predictors. __NOTE__ we have two targets here (`Y1, Y2`).

In [12]:
#y1 = data.Y1
#y2 = data.Y2

Check the shape of the data.

In [13]:
y2.shape

(660,)

In [14]:
y2.shape

(660,)

It is very possible that you will use different sets of the predictors for `Y1` and `Y2`. Now let's define them.

First, let's define predictors for `Y1` - which will be the first 5 features in `data`.

In [15]:
y1.value_counts()

 1.0    330
-1.0    330
Name: Y1, dtype: int64

In [16]:
y2.value_counts()

-0.788430    407
 1.268344    253
Name: Y2, dtype: int64

In [17]:
#cols = list(data.columns)
#cols2 = list(y2.columns)
# first 5 features 
# -3
#cols[:-3]

In [19]:
cols1[:-3]

['C3`', 'C1', 'C5', 'T1prime']

Use below code to select the first 5 features as predictors for `Y1`.

In [24]:
# -3
#predictors_y1 = data[cols[:-3]]
#predictors_y1.head()

,C2,S1prime_norm,C3_norm
0,1,-0.108850,0.852544
1,0,-1.352857,-1.422466
2,1,2.566039,-0.959449
3,1,2.011379,-0.482319
4,1,-0.343501,0.544111


Upon investigation of the data, we know we have __six__ features (`X1 - X6`) predicting `Y2`. Use similar code (as below) to select them.

In [33]:
# 
#predictors_y2 = data[cols[:-1]]
#predictors_y2.head()

,C2,C3_norm,C5_norm,C7_norm,T1prime
0,1,0.852544,-0.368850,-0.408144,0.171523
1,0,-1.422466,0.063268,-0.787037,0.233747
2,1,-0.959449,0.764109,-1.453046,0.119626
3,1,-0.482319,0.934891,-1.379812,-1.277595
4,1,0.544111,-0.015643,1.013623,-0.590665


Below is the key part of this notebook - which generates a `logistic regression` model to predict `Y1`/`Y2`.

The code works this way:

1. We generate two lists `f1_score_lst` and `auc_lst` to store f1_score and AUC from each of the `10` runs of the model;
2. Define model:
    1. We define a `LogisticRegression()` model;
    
    2. We split predictors (`predictors_y1`) and target `y1` to training (80%) and testing (20%);
    
    3. We fit the model `clf` to the training data, then use it to predict on the testing data;
    
    4. We also defined a `10-fold cross validation` to make sure our model do not overfit - see [here](https://scikit-learn.org/stable/modules/cross_validation.html) for more info;
    
    5. We append the f1_score and AUC of current model to the lists (`f1_score_lst` and `auc_lst`) we defined earlier.
  
3. Print out average f1_score and AUC for all 10 runs;
4. Print out average average accuracy from cross validation
5. Print out confusion matrix and classification report for the __last__ model.

__NOTE__: Step 3 provides the evaluation results we need; step 4 - 5 can be used to verify the results from step 3.

In [20]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf = LogisticRegression()
    X1_train, X1_test, y1_train, y1_test = train_test_split(predictors_y1, y1, test_size=0.2, random_state=123)
    clf.fit(X1_train, y1_train)

    y1_pred = clf.predict(X1_test)

    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf, X1_train, y1_train, cv=kfold, scoring=scoring)

    

    
    #calculate f1-score and AUC
    
    clf_roc_auc = roc_auc_score(y1_test, y1_pred)
    f1_score_lst.append(precision_recall_fscore_support(y1_test, y1_pred, average='weighted')[2])
    auc_lst.append(clf_roc_auc)


print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

#result=logit_model.fit()
confusion_matrix_y1 = confusion_matrix(y1_test, y1_pred)


#print(result.summary())
print('Accuracy of classifier on test set: {:.2f}'.format(clf.score(X1_test, y1_test)))

print("10-fold cross validation average accuracy of classifier: %.3f" % (results.mean()))

print('Confusion Matrix for Logistic Regression Classfier:')
print(confusion_matrix_y1)

print('Classification Report for Logistic Regression Classfier:')
print(classification_report(y1_test, y1_pred))


NameError: name 'predictors_y1' is not defined

Below code are used to evaluate model toward `Y2`. It is very similar to the code above - key difference is that `Y2` is imbalanced - so I wrote some code (under `# Begin oversampling`) to deal with that.

In [34]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf1 = LogisticRegression()

    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(predictors_y2, y2, test_size=0.2, random_state=123)
    
    # Begin oversampling
    oversample = pd.concat([X2_train,y2_train],axis=1)
    max_size = oversample['Y2'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('Y2'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['Y2'])
    del X2_train['Y2']
    
    # fitting model on oversampled data
    clf1.fit(X2_train, y2_train)
    
    y2_pred = clf1.predict(X2_test)
    
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))

/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is Fal

F1 0.5484; AUC 0.5644 
Accuracy of classifier on test set: 0.545
10-fold cross validation average accuracy of clf1: 0.600
Confusion Matrix for Classfier:
[[30 41]
 [19 42]]
Classification Report for Classfier:
              precision    recall  f1-score   support

           0       0.61      0.42      0.50        71
           1       0.51      0.69      0.58        61

    accuracy                           0.55       132
   macro avg       0.56      0.56      0.54       132
weighted avg       0.56      0.55      0.54       132

